In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from catboost import CatBoostClassifier

In [ ]:
train = pd.read_csv(r"/kaggle/input/playground-series-s5e12/train.csv")
test  = pd.read_csv(r"/kaggle/input/playground-series-s5e12/test.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)

In [ ]:
TARGET = "diagnosed_diabetes"

X = train.drop(columns=[TARGET])
y = train[TARGET]

print("X shape:", X.shape)
print("y shape:", y.shape)

assert len(X) == len(y)

In [ ]:
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
cat_idxs = [X.columns.get_loc(col) for col in cat_cols]

print("Categorical columns:")
print(cat_cols)

In [ ]:
N_SPLITS = 5

kf = StratifiedKFold(
    n_splits=N_SPLITS,
    shuffle=True,
    random_state=42
)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(test))

In [ ]:
for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f"\n========== Fold {fold+1} ==========")

    X_train, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    model = CatBoostClassifier(
        iterations=400,
        depth=5,
        learning_rate=0.07,
        loss_function="Logloss",
        eval_metric="AUC",
        random_seed=42,
        verbose=50
    )

    model.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        cat_features=cat_idxs,
        early_stopping_rounds=50
    )

    oof_preds[val_idx] = model.predict_proba(X_val)[:, 1]
    test_preds += model.predict_proba(test)[:, 1] / N_SPLITS

In [ ]:
print("Min:", test_preds.min())
print("Max:", test_preds.max())

In [ ]:
cv_auc = roc_auc_score(y, oof_preds)
print("\n==============================")
print("CatBoost CV AUC:", cv_auc)
print("==============================")

In [ ]:
submission = pd.read_csv(r"/kaggle/input/playground-series-s5e12/sample_submission.csv")

submission["diagnosed_diabetes"] = test_preds
submission.to_csv("submission.csv", index=False)

submission.head()